In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np

# Загрузка модели и токенизатора
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Загрузка файла CSV
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Чтение данных из CSV-файла с различными кодировками
try:
    df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='cp1252')
    except UnicodeDecodeError:
        df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='utf-16')

# Просмотр первых строк данных
df.head()

# Загрузка данных из CSV файла
df = pd.read_csv("situatsiya1.csv")

# Функция для генерации новой ситуации
def generate_situation(prompt, max_new_tokens=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Функция для генерации эмоции
def generate_emotion():
    emotions = df['first_transition_emotion'].tolist() + df['second_transition_emotion'].tolist()
    return np.random.choice(emotions)

# Функция для генерации новой частичной модели
def generate_partial_model(id):
    initial_situation = np.random.choice(df['initial_situation'].tolist())
    first_transition = generate_situation(f"После {initial_situation},")
    second_transition = generate_situation(f"После {first_transition},")

    return {
        "id": id,
        "initial_situation": initial_situation,
        "first_transition_situation": first_transition,
        "first_transition_emotion": generate_emotion(),
        "second_transition_situation": second_transition,
        "second_transition_emotion": generate_emotion()
    }

# Генерация 5 новых частичных моделей
new_models = [generate_partial_model(i) for i in range(1, 6)]
df_new_models = pd.DataFrame(new_models)
print(df_new_models)

# Функция для генерации спектра ситуаций с вероятностями
def generate_spectrum(initial_situation, num_transitions=3):
    transitions = []
    probabilities = np.random.dirichlet(np.ones(num_transitions))

    for i in range(num_transitions):
        transition = generate_situation(f"После {initial_situation},")
        emotion = generate_emotion()
        transitions.append({
            "transition_situation": transition,
            "transition_emotion": emotion,
            "probability": probabilities[i]
        })

    return transitions

# Генерация спектра ситуаций для одной из новых моделей
initial_situation = new_models[0]["initial_situation"]
spectrum = generate_spectrum(initial_situation)
print("\nСпектр ситуаций с вероятностями:")
for transition in spectrum:
    print(f"Ситуация: {transition['transition_situation']}")
    print(f"Эмоция: {transition['transition_emotion']}")
    print(f"Вероятность: {transition['probability']:.2f}")
    print()

# Сохранение новых моделей в CSV
df_new_models.to_csv("new_partial_models.csv", index=False)

In [ ]:
df.head()

,id,initial_situation,first_transition_situation,first_transition_emotion,second_transition_situation,second_transition_emotion
0,1,Покупка лотерейного билета,Выигрыш автомобиля,Радость,Путешествие на новом авто,Восторг
1,1,Покупка лотерейного билета,Проигрыш (ничего не выиграл),Разочарование,Попытка купить еще билетов,Надежда
2,2,Участие в марафоне,Получение медали,Радость,Празднование с друзьями,Восторг
3,2,Участие в марафоне,Неудача на финише,Разочарование,Участие в следующем марафоне,Надежда
4,3,Получение диплома,Начало новой работы,Восторг,Повышение на работе,Радость
